In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle

In [2]:
import mlflow
from mlflow.models.signature import infer_signature

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/07/19 15:19:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/19 15:19:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1752768385078, experiment_id='1', last_update_time=1752768385078, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[((df.duration >= 0) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.808922488244

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "ibrahim")
    
    mlflow.log_param("train-data", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:56:37] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.06968                           
[1]	validation-rmse:6.80375                           
[2]	validation-rmse:6.76403                           
[3]	validation-rmse:6.75698                           
[4]	validation-rmse:6.74889                           
[5]	validation-rmse:6.73690                           
[6]	validation-rmse:6.72907                           
[7]	validation-rmse:6.72365                           
[8]	validation-rmse:6.71273                           
[9]	validation-rmse:6.70754                           
[10]	validation-rmse:6.70284                          
[11]	validation-rmse:6.69865                          
[12]	validation-rmse:6.69617                          
[13]	validation-rmse:6.69329                          
[14]	validation-rmse:6.68818                          
[15]	validation-rmse:6.68477                          
[16]	validation-rmse:6.68158                          
[17]	validation-rmse:6.67821                          
[18]	valid

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:57:11] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.59985                                                   
[1]	validation-rmse:9.36591                                                    
[2]	validation-rmse:8.49683                                                    
[3]	validation-rmse:7.89858                                                    
[4]	validation-rmse:7.49098                                                    
[5]	validation-rmse:7.21533                                                    
[6]	validation-rmse:7.02903                                                    
[7]	validation-rmse:6.90372                                                    
[8]	validation-rmse:6.81520                                                    
[9]	validation-rmse:6.75307                                                    
[10]	validation-rmse:6.70844                                                   
[11]	validation-rmse:6.67593                                                   
[12]	validation-rmse:6.65032            

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:59:24] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.61094                                                       
[1]	validation-rmse:8.15518                                                       
[2]	validation-rmse:7.41915                                                       
[3]	validation-rmse:7.05462                                                       
[4]	validation-rmse:6.86677                                                       
[5]	validation-rmse:6.76088                                                       
[6]	validation-rmse:6.70261                                                       
[7]	validation-rmse:6.66698                                                       
[8]	validation-rmse:6.64297                                                       
[9]	validation-rmse:6.62714                                                       
[10]	validation-rmse:6.61609                                                      
[11]	validation-rmse:6.60358                                                      
[12]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:00:54] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.83946                                                       
[1]	validation-rmse:6.59137                                                       
[2]	validation-rmse:6.56698                                                       
[3]	validation-rmse:6.55421                                                       
[4]	validation-rmse:6.54071                                                       
[5]	validation-rmse:6.53855                                                       
[6]	validation-rmse:6.53038                                                       
[7]	validation-rmse:6.52302                                                       
[8]	validation-rmse:6.51778                                                       
[9]	validation-rmse:6.51133                                                       
[10]	validation-rmse:6.50568                                                      
[11]	validation-rmse:6.50138                                                      
[12]

KeyboardInterrupt: 

In [13]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:46:34] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.55159
[1]	validation-rmse:10.87339
[2]	validation-rmse:10.28049
[3]	validation-rmse:9.76513
[4]	validation-rmse:9.31911
[5]	validation-rmse:8.93448
[6]	validation-rmse:8.60356
[7]	validation-rmse:8.31779
[8]	validation-rmse:8.07522
[9]	validation-rmse:7.86438
[10]	validation-rmse:7.68520
[11]	validation-rmse:7.53306
[12]	validation-rmse:7.40266
[13]	validation-rmse:7.29338
[14]	validation-rmse:7.19871
[15]	validation-rmse:7.11803
[16]	validation-rmse:7.04824
[17]	validation-rmse:6.98933
[18]	validation-rmse:6.93818
[19]	validation-rmse:6.89337
[20]	validation-rmse:6.85581
[21]	validation-rmse:6.82331
[22]	validation-rmse:6.79425
[23]	validation-rmse:6.76834
[24]	validation-rmse:6.74749
[25]	validation-rmse:6.72835
[26]	validation-rmse:6.71171
[27]	validation-rmse:6.69665
[28]	validation-rmse:6.68405
[29]	validation-rmse:6.67181
[30]	validation-rmse:6.66018
[31]	validation-rmse:6.65007
[32]	validation-rmse:6.64198
[33]	validation-rmse:6.63520
[34]	validation-rmse:

In [ ]:
with mlflow.start_run():
     best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }
     
     mlflow.log_params(best_params)
     
     booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50)
     
     y_pred = booster.predict(valid)
     rmse = mean_squared_error(y_val, y_pred)
     mlflow.log_metric("rmse", rmse)
     
     with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
     
     mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
     
     mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:19:45] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.55159
[1]	validation-rmse:10.87339
[2]	validation-rmse:10.28049
[3]	validation-rmse:9.76513
[4]	validation-rmse:9.31911
[5]	validation-rmse:8.93448
[6]	validation-rmse:8.60356
[7]	validation-rmse:8.31779
[8]	validation-rmse:8.07522
[9]	validation-rmse:7.86438
[10]	validation-rmse:7.68520
[11]	validation-rmse:7.53306
[12]	validation-rmse:7.40266
[13]	validation-rmse:7.29338
[14]	validation-rmse:7.19871
[15]	validation-rmse:7.11803
[16]	validation-rmse:7.04824
[17]	validation-rmse:6.98933
[18]	validation-rmse:6.93818
[19]	validation-rmse:6.89337
[20]	validation-rmse:6.85581
[21]	validation-rmse:6.82331
[22]	validation-rmse:6.79425
[23]	validation-rmse:6.76834
[24]	validation-rmse:6.74749
[25]	validation-rmse:6.72835
[26]	validation-rmse:6.71171
[27]	validation-rmse:6.69665
[28]	validation-rmse:6.68405
[29]	validation-rmse:6.67181
[30]	validation-rmse:6.66018
[31]	validation-rmse:6.65007
[32]	validation-rmse:6.64198
[33]	validation-rmse:6.63520
[34]	validation-rmse:

2025/07/19 15:22:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:22:44] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/07/19 15:22:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [21]:
logged_model = "runs:/9765afabd7014ae78236cf0c30969ece/models_mlflow"
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [22]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-5caf4b22b91249f38ac1a9ad705f5b8f/artifacts
  flavor: mlflow.xgboost
  run_id: 9765afabd7014ae78236cf0c30969ece

In [23]:
import mlflow.xgboost


xgboost_model = mlflow.xgboost.load_model(logged_model)

In [24]:
xgboost_model

In [25]:
y_pred = xgboost_model.predict(valid)